# Beam search implementation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from queue import Queue

In [2]:
df_action = pd.read_csv('data/action_condition_meta.csv')
#df_action = df_action.drop('user_id', 1)
#df_action.drop_duplicates(subset = 'user_id', inplace = True)
df_action.head()

,action,user_id,condition,geo_country,refr_source,browser_language,os_name,os_timezone,dvce_type
0,clic,379881d5-32d7-49f4-bf5b-81fefbc5fcce,1-Control,FI,Google,greek,Android,Europe,Mobile
1,clic,2a0f4218-4f62-479b-845c-109b2720e6e7,2-Buttony-Conversion-Buttons,AU,Google,english,iOS,Australia,Mobile
2,clic,a511b6dc-2dca-455b-b5e2-bf2d224a5505,2-Buttony-Conversion-Buttons,GB,Google,english,Android,Europe,Mobile
3,clic,9fb616a7-4e13-4307-ac92-0b075d7d376a,2-Buttony-Conversion-Buttons,FI,Google,english,iOS,Europe,Mobile
4,clic,64816772-688d-4460-a591-79aa49bba0d5,2-Buttony-Conversion-Buttons,BD,Google,english,Android,Asia,Mobile


In [3]:
#created own priorityqueue class, because the standard priorityqueue blocks (can't insert element) when max.size is reached.
#we need to have a priorityqueue where the item with lowest priority is discarded and replaced by the new item.
#priorityqueue represents min heap 
import heapq
from heapq import heappush, heappop

class priority_queue:
    def __init__(self, max_size):
        self.items = []
        self.max = max_size
   
    def push(self, item, priority):
        if len(self.items) < self.max:
            heapq.heappush(self.items, (priority, item))
        else:
            heapq.heappushpop(self.items, (priority, item))
            

    def pop(self):
        return heapq.heappop(self.items)

    def get_max_item(self):
        return self.items[0]
    
    def empty(self):
        return not self.items
    
    def print_elements(self):
        result = []
        for i in self.items:
            result.append(i)
        return result
       
    def heap_sort(self):
        return [heapq.heappop(self.items) for _ in range(len(self.items))]

In [4]:
#desc = [('refr_source', 'Google'), ('geo_country', 'GB')]
#df_match_condition = create_dataframe(desc)
#constraints_2(df_match_condition)
#df_match_condition.shape

In [5]:
#desc = [('refr_source', 'Google'), ('geo_country', 'AZ')]
#phiYule(df_match_condition)

In [6]:
'''
bins = 3
language = []
The refinement operator. The refinement operator gets the records and chooses to which type of attribute they belong.
There are 3 types of attributes:
    1. Numeric: Attribute with all number records
    2. Binary: Attribute with true or false records
    3. Nominal: Attribute with multiple different values in their records which are not numeric
    
1. For numeric values we sort all records which are in the description language D. After that we make equal-sized bins.
The amount of bins is dependend on a predefined value. For each split point we add a description based on whether the
numeric value is greater or equal or smaller or equal as the split point.
2. For binary records we add one description where the description is true and one description where the description is false.
3. For nominal values we add for each description an entry with the description and one without the description. In our case
we take the first description in the list which is not equal to that description.
def refinement(seed):
    dataframe = create_dataframe(language)
    unique_records = dataframe[seed].unique()
    if unique_records.size == 0:
        return language
    print("uniques: ")
    print(unique_records)
    if isNumeric(unique_records):     #Attribute is numeric
        language.append(-1) #TODO       
    elif unique_records.size == 2:    #Attribute is binary
        language.append((seed, True))
        language.append((seed, False))
    else:                             #Attribute is nominal
        for item in unique_records:
            language.append((seed, item))
            language.append((seed, unique_records[next((i for i, v in enumerate(unique_records) if v != item), -1)]))
    return language

Checks whether the records are all numeric.

def isNumeric(unique_records):
    for item in unique_records:
        if not item.isdigit():
            return False
    return True

candidateQueue = list(df.columns.values) #instead of queue that is used in the paper, intialize the queue immediately
candidateQueue.remove('action') #with the headers of the dataframe
candidateQueue.remove('condition') #remove action and condition (targets) from this, because we only need the descriptors 

seed = candidateQueue.pop()
print("seed: " + seed)
print("lang:")
print(language)
print(refinement(seed))
print("ref result:")

seed = candidateQueue.pop()
print("seed: " + seed)
print("lang:")
print(language)
print(refinement(seed))
print("ref result:")
'''

'\nbins = 3\nlanguage = []\nThe refinement operator. The refinement operator gets the records and chooses to which type of attribute they belong.\nThere are 3 types of attributes:\n    1. Numeric: Attribute with all number records\n    2. Binary: Attribute with true or false records\n    3. Nominal: Attribute with multiple different values in their records which are not numeric\n    \n1. For numeric values we sort all records which are in the description language D. After that we make equal-sized bins.\nThe amount of bins is dependend on a predefined value. For each split point we add a description based on whether the\nnumeric value is greater or equal or smaller or equal as the split point.\n2. For binary records we add one description where the description is true and one description where the description is false.\n3. For nominal values we add for each description an entry with the description and one without the description. In our case\nwe take the first description in the list w

In [7]:
#def phiEntropy(Set): --> to be implemented!
#descLang = set()
descLang = []
numberOfBins = 3 #Should be used for numeric value refinement
'''
The refinement operator. The refinement operator gets the records and chooses to which type of attribute they belong.
There are 3 types of attributes:
    1. Numeric: Attribute with all number records
    2. Binary: Attribute with true or false records
    3. Nominal: Attribute with multiple different values in their records which are not numeric
    
1. For numeric values we sort all records which are in the description language D. After that we make equal-sized bins.
The amount of bins is dependend on a predefined value. For each split point we add a description based on whether the
numeric value is greater or equal or smaller or equal as the split point.
2. For binary records we add one description where the description is true and one description where the description is false.
3. For nominal values we add for each description an entry with the description and one without the description. In our case
we take the first description in the list which is not equal to that description.
'''
def refinement(seed):
    #dataframe = create_dataframe(descLang)
    unique_records = df_action[seed].unique()
    if unique_records.size == 0:
        return descLang
    if isNumeric(unique_records):     #Attribute is numeric
        descLang.append(-1) # TODO       
    elif unique_records.size == 2:    #Attribute is binary
        descLang.append((seed, True))
        descLang.append((seed, False))
    else:                             #Attribute is nominal
        for item in unique_records:
            descLang.append((seed, item))
            descLang.append((seed, unique_records[next((i for i, v in enumerate(unique_records) if v != item), -1)]))
    return descLang
'''
descList = []
def refinement(seed):
    descriptions = df_action[seed].unique()
    for item in descriptions:
        descList.append((seed, item))
    return descList
'''
'''
Checks whether a list of values is all numeric
'''
def isNumeric(values):
    for item in values:
        if not item.isdigit():
            return False
    return True

'''
Creates a dataframe filtered by descriptors
'''
def create_dataframe(Set):
    d =df_action.copy()
    count = 0
    for column, item in Set:
        if count==0:  
            df_new = pd.merge(d, df_action.loc[(df_action[column] == item)], on=list(df_action), how='inner')
            df_new.drop_duplicates(inplace = True)
        else:
            df_new = pd.merge(df_new, df_action.loc[(df_action[column] == item)], on=list(df_action), how='inner')
            df_new.drop_duplicates(inplace = True)
        count +=1
    return df_new

def constraints(df_matches):
    return df_matches.shape[0] > 5

def phiYule(df_matches):
    n_1 = df_matches.loc[(df_matches.action == 'view') & (df_matches.condition == '1-Control')].count()[0]
    n_2 = df_matches.loc[(df_matches.action == 'clic') & (df_matches.condition == '1-Control')].count()[0]
    n_3 = df_matches.loc[(df_matches.action == 'view') & (df_matches.condition == '2-Buttony-Conversion-Buttons')].count()[0]
    n_4 = df_matches.loc[(df_matches.action == 'clic') & (df_matches.condition == '2-Buttony-Conversion-Buttons')].count()[0]
    yuleQ = (n_1*n_4 - n_2*n_3)/(n_1*n_4 + n_2*n_3)
    return yuleQ

def beam_search(d, w, q):
    candidateQueue = list(df_action.columns.values) #instead of queue that is used in the paper, intialize the queue immediately
    candidateQueue.remove('action') #with the headers of the dataframe
    candidateQueue.remove('condition') #remove action and condition (targets) from this, because we only need the descriptors 
    candidateQueue.remove('user_id')
    #pseudo-code below is in line with the paper
    resultSet = priority_queue(q)
    for level in range(0, d):
        print("level: "+str(level))
        beam = priority_queue(w)
        while (len(candidateQueue) > 0):
            seed = candidateQueue.pop(0)
            set_refined = list(set(refinement(seed)))
            for desc in set_refined:
                df_match_condition = create_dataframe([desc])
                quality = phiYule(df_match_condition)
                if constraints(df_match_condition):
                    resultSet.push(desc, quality)
                    beam.push(desc, quality)
            while not beam.empty:
                candidateQueue.append(priority_queue.pop(beam))
    return resultSet

#result = beam_search(d=2, w=5, q=5)
#priority_queue.heap_sort(result)

In [8]:
descList = [[('refr_source', 'Google'), ('geo_country', 'GB')], [('refr_source', 'StudyPortal'), ('geo_country', 'HK')]]
for item in descList:
    df_match_condition = create_dataframe(item)
    quality = phiYule(df_match_condition)
    print("Yule's Quality of "+str(item)+" = "+str(quality))

Yule's Quality of [('refr_source', 'Google'), ('geo_country', 'GB')] = 0.501340482574
Yule's Quality of [('refr_source', 'StudyPortal'), ('geo_country', 'HK')] = 1.0


In [75]:

d = 2
descLang = [[] for _ in range(len(df_action))]
def refinement_2(seed):
    for level in range(1,d+1):
        #dataframe = create_dataframe(descLang)
        unique_records = df_action[seed].unique()
        if unique_records.size == 0:
            return descLang
        if isNumeric(unique_records):     #Attribute is numeric
            descLang.append(-1) # TODO       
        elif unique_records.size == 2:    #Attribute is binary
            descLang.append((seed, True))
            descLang.append((seed, False))
        else:                             #Attribute is nominal
            for item in unique_records:
                #print(item)
                index = 0
                for item2 in unique_records:
                    #print(item)
                    if (level == 1):
                        descLang[index].append((seed, item))
                        index += 1
                    else:
                        descLang[index].append((seed, unique_records[next((i for i, v in enumerate(unique_records) if v != item2), -1)]))
                        #descLang[index].append((seed, item2))
                        index += 1

        return descLang

In [80]:
candidateQueue = list(df_action.columns.values) #instead of queue that is used in the paper, intialize the queue immediately
candidateQueue.remove('action') #with the headers of the dataframe
candidateQueue.remove('condition') #remove action and condition (targets) from this, because we only need the descriptors 
candidateQueue.remove('user_id')
candidateQueue.remove('geo_country')
candidateQueue.remove('refr_source')
candidateQueue.remove('browser_language')
candidateQueue.remove('os_timezone')
#candidateQueue.remove('dvce_type')
seed = candidateQueue.pop(0)
set_refined = refinement_2(seed)
set_refined = [x for x in set_refined if x != []]
result = [list(i) for i in set(tuple(row) for row in set_refined)]
result
result

[[('os_name', 'Android'),
  ('os_name', 'iOS'),
  ('os_name', 'Other'),
  ('os_name', 'Windows'),
  ('os_name', 'Android'),
  ('os_name', 'iOS'),
  ('os_name', 'Other'),
  ('os_name', 'Windows'),
  ('os_name', 'Android'),
  ('os_name', 'iOS'),
  ('os_name', 'Other'),
  ('os_name', 'Windows')]]

In [78]:
descList = [[('refr_source', 'Google'), ('geo_country', 'GB')], [('refr_source', 'StudyPortal'), ('geo_country', 'HK')]]
for item in descList:
    df_match_condition = create_dataframe(item)
    quality = phiYule(df_match_condition)
    #print("Yule's Quality of "+str(item)+" = "+str(quality))
    print(str(quality))

0.501340482574
1.0
